<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
try: 
  drive.mount('/content/drive/', force_remount = True)
  COLAB = True
  import tensorflow as tf
  print(f"You are on Google Colab with tensorflow version {tf.__version__}")
except:
  COLAB = False
  print("You are not connected")

Mounted at /content/drive/
You are on Google Colab with tensorflow version 2.3.0


In [31]:
def timeset(x):
  hrs = int(x/(60 *60))
  mnt = int(x%(60 *60)/60)
  secn = int(x%60)
  return f"{hrs}: {mnt:>02}: {secn:>05.2f}"

In [32]:
import time
import statistics
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, PReLU, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [33]:
emp = pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv", na_values = ['NAN','?'])

In [34]:
display(emp.head())

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [35]:
emp.drop('id', axis = 1, inplace = True)

In [36]:
emp['income'] = emp['income'].fillna(emp['income'].median())

In [37]:
contcols = emp.columns.drop(['job','area','product','subscriptions'])

In [38]:
y = emp['product']

In [39]:
emp.drop('product', axis = 1, inplace = True)

In [40]:
aread = pd.get_dummies(emp['area'], prefix = 'area')

In [41]:
jobd = pd.get_dummies(emp['job'], prefix = 'job')

In [42]:
emp.drop(['job', 'area'], axis = 1, inplace = True)

In [43]:
display(emp.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime
0,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100
1,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809
2,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723
3,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216
4,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033


In [44]:
for col in contcols:
  emp[col] = zscore(emp[col])

In [45]:
display(emp.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime
0,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315
1,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945
2,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087
3,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401
4,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090


In [46]:
emp = pd.concat([emp, aread, jobd], axis = 1)

In [47]:
display(emp.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,area_a,area_b,area_c,area_d,job_11,job_al,job_am,job_ax,job_bf,job_by,job_cv,job_de,job_dz,job_e2,job_f8,job_gj,job_gv,job_kd,job_ke,job_kl,job_kp,job_ks,job_kw,job_mm,job_nb,job_nn,job_ob,job_pe,job_po,job_pq,job_pz,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz
0,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
x = emp.values

In [49]:
y = pd.get_dummies(y).values

In [50]:
x[0:10]

array([[-0.60754957, -0.66491815,  1.        , -0.04841068, -0.21576413,
        -0.31408854,  0.85432106,  0.07927915, -0.46576475, -1.12031509,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.33805295, -0.20774798,  2.        , -0.26676549,  0.19686897,
        -0.91516076,  1.39443237, -0.07501047, -1.44537236,  0.68294487,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.     

In [51]:
y[0:10]

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [30]:
#Model setup

In [56]:
def my_turned_model(dropout, lr,neuronRate, neuronDrop):
  SPLITS = 10
  boots_samples = StratifiedShuffleSplit(n_splits = SPLITS, test_size= 0.1)
  average_epoch = []
  average_lloss = []
  count = 0
  no_neurons = (neuronRate * 8000)
  
  #Looping over the bootstraps samples
  for train, test in boots_samples.split(x, y):
    begin1 = time.time()
    count+=1
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    #Model setup
    layers = 0
    model = Sequential()
    while layers < 10 and no_neurons > 40:
      if layers == 0:
        model.add(Dense(no_neurons, input_dim = x.shape[1], activation = PReLU(), kernel_initializer = 'random_normal'))
      else:
        model.add(Dense(no_neurons, activation = PReLU()))
      no_neurons = (no_neurons * neuronRate)
      model.add(Dropout(dropout))
      model.add(Dense(y.shape[1], activation = 'softmax'))
      model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr))
      info = EarlyStopping(monitor = 'val_loss', patience = 100, verbose = 1, min_delta = 1e-3, restore_best_weights = True)
      model.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = True, epochs = 1000, callbacks = [info])
      epoc = info.stopped_epoch
      average_epoch.append(epoc)
      pred = model.predict(x_test)
      y_match = np.argmax(y_test, axis = 1)
      scoreloss = metrics.log_loss(y_test, pred)
      average_lloss.append(scoreloss)
      mean1 = statistics.mean(average_epoch)
      mean2 = statistics.mean(average_lloss)
      mean_dev = statistics.pstdev(average_lloss)
    end1 = time.time()
    total_time = begin1 - end1
    total_time = timeset(total_time)
    tensorflow.keras.backend.clear_session()
    return f"Minus Mean_score_logloss: {mean2}\n Average Epoch :{mean1}\n Mean_deviation_loss: {mean_dev}, Time elapsed: {total_time}"
print(my_turned_model(dropout = 0.25,lr = 1e-03,neuronRate = 0.2, neuronDrop = 0.25))


Epoch 1/1000
57/57 [==============================] - 0s 4ms/step - loss: 1.0376 - val_loss: 0.7584
Epoch 2/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.7589 - val_loss: 0.6823
Epoch 3/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.6883 - val_loss: 0.6655
Epoch 4/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.6469 - val_loss: 0.6429
Epoch 5/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.6250 - val_loss: 0.6070
Epoch 6/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.6092 - val_loss: 0.6262
Epoch 7/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.5836 - val_loss: 0.6043
Epoch 8/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.5708 - val_loss: 0.6026
Epoch 9/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.5646 - val_loss: 0.6143
Epoch 10/1000
57/57 [==============================] - 0s 3ms/step - loss: 0.5375 - val_loss: 0.6152

In [59]:
#Model Hyperparameters turning's generator
def helper_hyper(dropout, neuronpct,neuronCut):
  num_neurons = (6000 * neuronpct)
  layers = 0
  model = Sequential()
  while layers < 10 and num_neurons > 50:
    if layers == 0:
      model.add(Dense(num_neurons, input_dim = x.shape[1], activation = PReLU(), kernel_initializer = 'random_normal'))
    else:
      model.add(Dense(num_neurons, activation = PReLU(), kernel_initializer = 'random_normal'))
    layers+=1
    num_neurons = (num_neurons * neuronCut)
    model.add(Dropout(dropout))
    model.add(Dense(y.shape[1], activation = 'softmax'))
  return model

In [63]:
mymodel = helper_hyper(dropout = 0.20, neuronpct=0.1,neuronCut=0.20)

In [64]:
mymodel.summary()#Display out the blueprint

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 600)               29400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 4207      
_________________________________________________________________
dense_7 (Dense)              (None, 120)               1080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 847       
Total params: 35,534
Trainable params: 35,534
Non-trainable params: 0
__________________________________________________

In [68]:
#Create the model turned using the above helper
def myturned_nnt(dropout, lr, neuronsRate,neuronsCut):
  SPLITS = 10
  boots_samples = StratifiedShuffleSplit(n_splits = SPLITS, test_size = 0.1)
  average_log_loss = []
  average_epoch = []
  count = 0
  neurons_nums = int(neuronsRate * 8000)
  #Looping through the bootstrap samples:
  for train, test in boots_samples.split(x, y):
    time_start = time.time()
    count+=1
    x_train = x[train]
    x_test = x[test]
    y_train = y[train]
    y_test = y[test]

    #Generating the model using the above helper function
    model = helper_hyper(dropout, neuronsRate, neuronsCut)
    model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr=lr))
    info = EarlyStopping(monitor = 'val_loss', patience = 100, verbose = 1, restore_best_weights = True, min_delta = 1e-3)
    model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 1000, verbose = 0, callbacks = [info])
    epoc1 = info.stopped_epoch
    average_epoch.append(epoc1)
    pred = model.predict(x_test)
    y_match = np.argmax(y_test, axis = 1)
    scoreloss = metrics.log_loss(y_match, pred)
    average_log_loss.append(scoreloss)
    time_end = time.time()
    time_elapsed = time_start - time_end
  tensorflow.keras.backend.clear_session()
  return f"Average_logloss: {statistics.mean(average_log_loss)}\n Average epochs: {statistics.mean(average_epoch)}\n Time elapsed: {time_elapsed}"

print(myturned_nnt(dropout = 0.2, lr = 1e-03, neuronsRate = 0.1, neuronsCut = 0.25))

Restoring model weights from the end of the best epoch.
Epoch 00133: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00125: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00130: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00117: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00116: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00115: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00121: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00130: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00112: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00127: early stopping
Average_logloss: 0.7259871798576787
 Average epochs: 121.6
 Time elapsed: -16.94886016845703
